In [1]:
import pymysql
import pandas as pd

In [2]:
# Import statements
import sys
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Conv1D, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model

from keras import initializers, regularizers, constraints, optimizers, layers
import time

plt.style.use(style="seaborn")
%matplotlib inline



In [3]:
#connecting to MySQL
conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='*****',db='test')

df=pd.read_sql_query("SELECT * FROM chat ",conn)


print(df)

   user_id  thread_id  id                                               body
0        1        101   1               I am glad to be a part of this team.
1        2        102   2  That approach could me made easier if we use t...
2        1        103   3                I agree and I totally support that.
3        3        101   4  Let us focus on this first and then move on to...
4        3        105   5  How about implementing this uisng that model a...
5        4        106   6                 we could for more insights on this
6        4        107   7  Two of us could work on the first part and the...


In [4]:
#Grouping data based on user_id
conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='*****',db='test')
test=pd.read_sql_query("select user_id,group_concat(body) as TEXT from chat group by user_id ",conn)
print(test)

   user_id                                               TEXT
0        1  I am glad to be a part of this team.,I agree a...
1        2  That approach could me made easier if we use t...
2        3  Let us focus on this first and then move on to...
3        4  we could for more insights on this,Two of us c...


In [5]:
#Combining MBTI and Big-Five Models
def mbti_to_big5(mbti):
   
  
    mbti = mbti.lower()
    cEXT, cNEU, cAGR, cCON, cOPN = 0,np.NaN,0,0,0
    if mbti[0] == "i":
        cEXT = 0
    elif mbti[0] == "e":
        cEXT = 1

    
    if mbti[1] == "n":
        cOPN = 1
    elif mbti[1] == "s":
        cOPN = 0   

    
    if mbti[2] == "t":
        cAGR = 0
    elif mbti[2] == "f":
        cAGR = 1

    if mbti[3] == "p":
        cCON = 0
    elif mbti[3] == "j":
        cCON = 1

    return cEXT, cNEU, cAGR, cCON, cOPN

In [6]:
df_essays =pd.read_csv('essays.csv', encoding='cp1252', delimiter=',', quotechar='"')

In [7]:
for e in df_essays.columns[2:7]:
    df_essays[e] = df_essays[e].replace('n', '0')
    df_essays[e] = df_essays[e].replace('y', '1')
    
    df_essays[e] = pd.to_numeric(df_essays[e])

In [8]:
df_essays.head()

,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,1997_504851.txt,"Well, right now I just woke up from a mid-day ...",0,1,1,0,1
1,1997_605191.txt,"Well, here we go with the stream of consciousn...",0,0,1,0,0
2,1997_687252.txt,An open keyboard and buttons to push. The thin...,0,1,0,1,1
3,1997_568848.txt,I can't believe it! It's really happening! M...,1,0,1,1,0
4,1997_688160.txt,"Well, here I go with the good old stream of co...",1,0,1,0,1


In [9]:
test.head()

,user_id,TEXT
0,1,"I am glad to be a part of this team.,I agree a..."
1,2,That approach could me made easier if we use t...
2,3,Let us focus on this first and then move on to...
3,4,"we could for more insights on this,Two of us c..."


In [10]:
df_kaggle = pd.read_csv('mbti_1.csv',  skiprows=0 )

df_kaggle["cEXT"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[0], 1)
df_kaggle["cNEU"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[1], 1)
df_kaggle["cAGR"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[2], 1)
df_kaggle["cCON"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[3], 1)
df_kaggle["cOPN"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[4], 1)

df_kaggle = df_kaggle[["posts", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]]
df_kaggle.columns = ["TEXT", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]

# remove |||
df_kaggle["TEXT"] = df_kaggle.apply(lambda x: x.TEXT.replace("|||", " ")[:], 1)


df_kaggle

,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,'http://www.youtube.com/watch?v=qsXHcwe3krw ht...,0,NaN,1,1,1
1,'I'm finding the lack of me in these posts ver...,1,NaN,0,0,1
2,'Good one _____ https://www.youtube.com/wat...,0,NaN,0,0,1
3,"'Dear INTP, I enjoyed our conversation the o...",0,NaN,0,1,1
4,'You're fired. That's another silly misconcept...,1,NaN,0,1,1
...,...,...,...,...,...,...
8670,'https://www.youtube.com/watch?v=t8edHB_h908 I...,0,NaN,1,0,0
8671,'So...if this thread already exists someplace ...,1,NaN,1,0,1
8672,'So many questions when i do these things. I ...,0,NaN,0,0,1
8673,'I am very conflicted right now when it comes ...,0,NaN,1,0,1


In [11]:
#concatinate the dataframes:
frames  = [df_essays, df_kaggle]
essay = pd.concat(frames, sort=False)
essay.reset_index(drop=True)




print("saved entries: ", len(essay))

saved entries:  11142


In [12]:
essay

,#AUTHID,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,1997_504851.txt,"Well, right now I just woke up from a mid-day ...",0,1.0,1,0,1
1,1997_605191.txt,"Well, here we go with the stream of consciousn...",0,0.0,1,0,0
2,1997_687252.txt,An open keyboard and buttons to push. The thin...,0,1.0,0,1,1
3,1997_568848.txt,I can't believe it! It's really happening! M...,1,0.0,1,1,0
4,1997_688160.txt,"Well, here I go with the good old stream of co...",1,0.0,1,0,1
...,...,...,...,...,...,...,...
8670,NaN,'https://www.youtube.com/watch?v=t8edHB_h908 I...,0,NaN,1,0,0
8671,NaN,'So...if this thread already exists someplace ...,1,NaN,1,0,1
8672,NaN,'So many questions when i do these things. I ...,0,NaN,0,0,1
8673,NaN,'I am very conflicted right now when it comes ...,0,NaN,1,0,1


In [13]:
#  removing cNEU since it's not avaiable in MTBI model
essay.drop('cNEU', inplace=True, axis=1) 

In [14]:
essay

,#AUTHID,TEXT,cEXT,cAGR,cCON,cOPN
0,1997_504851.txt,"Well, right now I just woke up from a mid-day ...",0,1,0,1
1,1997_605191.txt,"Well, here we go with the stream of consciousn...",0,1,0,0
2,1997_687252.txt,An open keyboard and buttons to push. The thin...,0,0,1,1
3,1997_568848.txt,I can't believe it! It's really happening! M...,1,1,1,0
4,1997_688160.txt,"Well, here I go with the good old stream of co...",1,1,0,1
...,...,...,...,...,...,...
8670,NaN,'https://www.youtube.com/watch?v=t8edHB_h908 I...,0,1,0,0
8671,NaN,'So...if this thread already exists someplace ...,1,1,0,1
8672,NaN,'So many questions when i do these things. I ...,0,0,0,1
8673,NaN,'I am very conflicted right now when it comes ...,0,1,0,1


Remove URLs and HTML

In [15]:
import re


def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

In [16]:
#remove emojis
def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

Remove punctuation

In [17]:
import string


def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [18]:
essay["TEXT"] = essay.TEXT.map(lambda x: remove_URL(x))
essay["TEXT"] = essay.TEXT.map(lambda x: remove_html(x))
essay["TEXT"] = essay.TEXT.map(lambda x: remove_emoji(x))
essay["TEXT"] = essay.TEXT.map(lambda x: remove_punct(x))

In [19]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Remove stopwords

In [20]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))


def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [21]:
essay["TEXT"] = essay["TEXT"].map(remove_stopwords)

In [22]:
essay.TEXT 

0       well right woke midday nap sort weird ever sin...
1       well go stream consciousness essay used things...
2       open keyboard buttons push thing finally worke...
3       cant believe really happening pulse racing lik...
4       well go good old stream consciousness assignme...
                              ...                        
8670    ixfp always think cats fi doms reason especial...
8671    soif thread already exists someplace else heck...
8672    many questions things would take purple pill p...
8673    conflicted right comes wanting children honest...
8674    long since personalitycafe although doesnt see...
Name: TEXT, Length: 11142, dtype: object

In [23]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
from nltk.tokenize import word_tokenize



def create_corpus_tk(df):
    corpus = []
    for text in essay["TEXT"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [25]:
corpus = create_corpus_tk(essay)

In [26]:
num_words = len(corpus)
print(num_words)

11142


In [27]:
#corpus[0]

Train / test split

In [28]:
train_size = int(essay.shape[0] * 0.8)

train_sentences = essay.TEXT[:train_size]
train_labels_cEXT = essay.cEXT[:train_size]
train_labels_cAGR = essay.cAGR[:train_size]
train_labels_cCON = essay.cCON[:train_size]
train_labels_cOPN = essay.cOPN[:train_size]

test_sentences = essay.TEXT[train_size:]
test_labels_cEXT = essay.cEXT[train_size:]
test_labels_cAGR = essay.cAGR[train_size:]
test_labels_cCON = essay.cCON[train_size:]
test_labels_cOPN = essay.cOPN[train_size:]

In [29]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_len = 200

In [30]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_sentences)

In [31]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)

In [32]:
train_sentences

0       well right woke midday nap sort weird ever sin...
1       well go stream consciousness essay used things...
2       open keyboard buttons push thing finally worke...
3       cant believe really happening pulse racing lik...
4       well go good old stream consciousness assignme...
                              ...                        
6441    im sure whats better posting individual pictur...
6442    im well second year didnt finish last year im ...
6443    agree except jessica chastain believe enfj hig...
6444    always take much think reasonable wham never t...
6445    wow dude really spoton im energized people get...
Name: TEXT, Length: 8913, dtype: object

In [109]:
train_sequences

In [34]:
train_padded = pad_sequences(
    train_sequences, maxlen=max_len, truncating="post", padding="post"
)

In [35]:
train_padded

array([[  15,   37, 1948, ...,  172,   31, 2299],
       [  15,   23, 1260, ...,  285, 1036,    4],
       [ 369, 2349, 4717, ...,  423,  423,  106],
       ...,
       [ 135,  467, 5594, ...,   75,  828,  614],
       [  27,   60,   13, ..., 1066,  398, 1509],
       [ 322, 1019,    6, ...,   31, 1621, 8539]])

In [36]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(
    test_sequences, maxlen=max_len, padding="post", truncating="post"
)

In [37]:
test_padded

array([[  17,   21, 1199, ...,  419, 1737, 1279],
       [ 664,  514,  943, ..., 3807, 1675, 4694],
       [4237,  796,  126, ...,   78, 2796,  103],
       ...,
       [  56,  341,   18, ...,  307,  198,   34],
       [6614,   37,  238, ...,  760,   46,   11],
       [  83,   87, 5396, ...,  608,   14,   79]])

In [38]:
#print(essay.TEXT[0])
#print(train_sequences[0])

In [39]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 140867


In [110]:
word_index

Create the Embedding dictionary

In [41]:
embedding_dict = {}
with open("glove.6B.50d.txt", "r",encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [111]:
embedding_dict

In [43]:
#create embedding matrix
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 50))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [44]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.067678  ,  0.51832002,  1.32599998, ..., -0.65103   ,
         0.12924001,  0.48723999],
       [ 0.36807999,  0.20834   , -0.22318999, ..., -0.67236   ,
        -0.39708999,  0.25183001],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.02339995, -0.50840002, -1.67120004, ..., -1.65760005,
        -0.31445   ,  0.14918   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [45]:
print(train_padded.shape)
print(train_labels_cEXT.shape)

(8913, 200)
(8913,)


In [46]:
print(test_padded.shape)
print(test_labels_cEXT.shape)

(2229, 200)
(2229,)


In [47]:
import tensorflow as tf

In [48]:
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam

from keras.layers import InputLayer, Activation, merge, Concatenate,Input

In [49]:
# Bidirectional LSTM Model
# Input Layer
ip = Input(shape=(max_len, ))

# Embedding Layer
x = Embedding(num_words, 50, weights=[embedding_matrix],input_length=max_len, trainable=False)(ip)

# Bidirectional LSTM Layer 
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))(x) 
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

# Average and Max Pooling --> To reduce dimensionalty

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

# Final Dense Layer + activation(sigmoid)
x = Dense(1, activation='sigmoid')(x)




In [50]:
# Define model and optimization process
model1 = Model(inputs=ip, outputs=x)
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size = 32
epochs = 3
model1.fit(train_padded,train_labels_cEXT,batch_size=batch_size, epochs=epochs,validation_data=(test_padded, test_labels_cEXT), verbose=1)



model1.summary()

Epoch 1/3
279/279 [==============================] - 747s 3s/step - loss: 0.6066 - accuracy: 0.6910 - val_loss: 0.5235 - val_accuracy: 0.7734
Epoch 2/3
279/279 [==============================] - 568s 2s/step - loss: 0.5866 - accuracy: 0.6938 - val_loss: 0.5370 - val_accuracy: 0.7716
Epoch 3/3
279/279 [==============================] - 331s 1s/step - loss: 0.5639 - accuracy: 0.7132 - val_loss: 0.5240 - val_accuracy: 0.7734
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 50)      7043400     input_1[0][0]                    
________________________________________________________________

In [51]:
# Define model and optimization process
model3 = Model(inputs=ip, outputs=x)
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size = 32
epochs = 3
model3.fit(train_padded,train_labels_cAGR,batch_size=batch_size, epochs=epochs,validation_data=(test_padded, test_labels_cAGR), verbose=1)

model3.summary()

Epoch 1/3
279/279 [==============================] - 358s 1s/step - loss: 0.6961 - accuracy: 0.5769 - val_loss: 0.6394 - val_accuracy: 0.6406
Epoch 2/3
279/279 [==============================] - 350s 1s/step - loss: 0.6304 - accuracy: 0.6455 - val_loss: 0.6013 - val_accuracy: 0.6792
Epoch 3/3
279/279 [==============================] - 347s 1s/step - loss: 0.6037 - accuracy: 0.6752 - val_loss: 0.6071 - val_accuracy: 0.6747
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 50)      7043400     input_1[0][0]                    
______________________________________________________________

In [52]:
# Define model and optimization process
model4 = Model(inputs=ip, outputs=x)
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size = 32
epochs = 3
model4.fit(train_padded,train_labels_cCON,batch_size=batch_size, epochs=epochs,validation_data=(test_padded, test_labels_cCON), verbose=1)

model4.summary()


Epoch 1/3
279/279 [==============================] - 361s 1s/step - loss: 0.6851 - accuracy: 0.5871 - val_loss: 0.6611 - val_accuracy: 0.6057
Epoch 2/3
279/279 [==============================] - 353s 1s/step - loss: 0.6357 - accuracy: 0.6357 - val_loss: 0.6608 - val_accuracy: 0.6209
Epoch 3/3
279/279 [==============================] - 348s 1s/step - loss: 0.6225 - accuracy: 0.6505 - val_loss: 0.6718 - val_accuracy: 0.6196
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 50)      7043400     input_1[0][0]                    
______________________________________________________________

In [53]:
# Define model and optimization process
model5 = Model(inputs=ip, outputs=x)
model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
batch_size = 32
#epochs = 3
model5.fit(train_padded, train_labels_cOPN,batch_size=batch_size, epochs=3,validation_data=(test_padded, test_labels_cOPN), verbose=1)

model5.summary()

Epoch 1/3
279/279 [==============================] - 348s 1s/step - loss: 0.5367 - accuracy: 0.7535 - val_loss: 0.3902 - val_accuracy: 0.8609
Epoch 2/3
279/279 [==============================] - 338s 1s/step - loss: 0.4489 - accuracy: 0.7954 - val_loss: 0.3885 - val_accuracy: 0.8596
Epoch 3/3
279/279 [==============================] - 373s 1s/step - loss: 0.4453 - accuracy: 0.7993 - val_loss: 0.3887 - val_accuracy: 0.8609
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 50)      7043400     input_1[0][0]                    
______________________________________________________________

In [86]:
sequences = tokenizer.texts_to_sequences(test.TEXT)
padded = pad_sequences(sequences, maxlen=max_len, padding="post", truncating="post")

In [87]:
pred = model1.predict(padded)
pred_int = pred.round().astype("int")

In [88]:
pred4 = model4.predict(padded)
pred_int4 = pred4.round().astype("int")

In [89]:
pred3 = model3.predict(padded)
pred_int3 = pred3.round().astype("int")

In [90]:
pred5 = model5.predict(padded)
pred_int5 = pred5.round().astype("int")

In [91]:
pred

array([[0.41134688],
       [0.57806814],
       [0.42836696],
       [0.5288849 ]], dtype=float32)

In [92]:
pred3

array([[0.41134688],
       [0.57806814],
       [0.42836696],
       [0.5288849 ]], dtype=float32)

In [93]:
pred4

array([[0.41134688],
       [0.57806814],
       [0.42836696],
       [0.5288849 ]], dtype=float32)

In [94]:
pred5

array([[0.41134688],
       [0.57806814],
       [0.42836696],
       [0.5288849 ]], dtype=float32)

In [95]:
test['cEXT'] = pred_int

In [97]:
test['cAGR'] = pred_int3

In [99]:
test['cCON'] = pred_int4

In [101]:
test['cOPN'] = pred_int5

In [108]:
test.head(25)

,user_id,TEXT,cEXT,cAGR,cCON,cOPN
0,1,"I am glad to be a part of this team.,I agree a...",0,0,0,0
1,2,That approach could me made easier if we use t...,1,1,1,1
2,3,Let us focus on this first and then move on to...,0,0,0,0
3,4,"we could for more insights on this,Two of us c...",1,1,1,1


In [74]:
test.to_csv('final2.csv', index=False)